In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from pandas import concat
from pandas import read_csv
from helper import series_to_supervised, stage_series_to_supervised
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Dropout, concatenate

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0, 7"

### Dataset

In [3]:
dataset = pd.read_csv('../../data/Merged-update_hourly.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [4]:
# data.tail(10)

### Dataset Information

In [5]:
print(list(data.columns))

['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN']


### Pre-processing

In [6]:
# specify the number of lag hours
n_hours = 24*3
K = 24
n_features = data.shape[1]

#### Stage

In [7]:
# Pre-processing
# Stage --> 8 stages
stages = dataset[['WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26']]
print("stages.shape:", stages.shape)

# stages_supervised = stage_series_to_supervised(stages, n_hours, K, 1)
stages_supervised = series_to_supervised(stages, n_hours, K)
stages_supervised.reset_index(drop=True, inplace=True)
print("stages_supervised.shape:", stages_supervised.shape)

stages.shape: (96432, 4)
stages_supervised.shape: (96337, 384)


In [8]:
# stages_supervised

#### Prior unknown covariates (6)

In [9]:
prior_unknown_cov = data[['FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26' ]]
print("prior_unknown_cov.shape:", prior_unknown_cov.shape)

prior_unknown_cov_supervised = series_to_supervised(prior_unknown_cov, n_hours, K)
print("prior_unknown_cov_supervised.shape:", prior_unknown_cov_supervised.shape)

prior_unknown_cov.shape: (96432, 6)
prior_unknown_cov_supervised.shape: (96337, 576)


In [10]:
prior_unknown_cov_supervised.reset_index(drop=True, inplace=True)
prior_unknown_cov_supervised.columns = ['past_unknown_cov_' + i for i in list(prior_unknown_cov_supervised.columns)]

In [11]:
prior_unknown_cov_supervised

,past_unknown_cov_var1(t-72),past_unknown_cov_var2(t-72),past_unknown_cov_var3(t-72),past_unknown_cov_var4(t-72),past_unknown_cov_var5(t-72),past_unknown_cov_var6(t-72),past_unknown_cov_var1(t-71),past_unknown_cov_var2(t-71),past_unknown_cov_var3(t-71),past_unknown_cov_var4(t-71),...,past_unknown_cov_var3(t+22),past_unknown_cov_var4(t+22),past_unknown_cov_var5(t+22),past_unknown_cov_var6(t+22),past_unknown_cov_var1(t+23),past_unknown_cov_var2(t+23),past_unknown_cov_var3(t+23),past_unknown_cov_var4(t+23),past_unknown_cov_var5(t+23),past_unknown_cov_var6(t+23)
0,0.0,2.17,0.00,2.84,0.00,2.78,0.0,2.17,0.00,2.84,...,0.0,2.88,0.00,2.72,0.0,2.10,0.0,2.89,0.00,2.73
1,0.0,2.17,0.00,2.84,0.00,2.78,0.0,2.17,0.00,2.84,...,0.0,2.89,0.00,2.73,0.0,2.12,0.0,2.87,0.00,2.74
2,0.0,2.17,0.00,2.84,0.00,2.79,0.0,2.15,0.00,2.83,...,0.0,2.87,0.00,2.74,0.0,2.14,0.0,2.89,0.00,2.74
3,0.0,2.15,0.00,2.83,0.00,2.79,0.0,2.13,0.00,2.84,...,0.0,2.89,0.00,2.74,0.0,2.15,0.0,2.87,0.00,2.74
4,0.0,2.13,0.00,2.84,0.00,2.79,0.0,2.12,0.00,2.84,...,0.0,2.87,0.00,2.74,0.0,2.15,0.0,2.89,0.00,2.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96332,0.0,1.92,240.09,2.52,302.67,2.55,0.0,1.93,325.17,2.51,...,0.0,2.83,534.14,2.47,0.0,1.92,0.0,2.86,456.34,2.49
96333,0.0,1.93,325.17,2.51,393.59,2.48,0.0,1.94,406.56,2.48,...,0.0,2.86,456.34,2.49,0.0,1.92,0.0,2.88,380.19,2.48
96334,0.0,1.94,406.56,2.48,475.32,2.40,0.0,1.94,445.67,2.43,...,0.0,2.88,380.19,2.48,0.0,1.93,0.0,2.89,287.12,2.54
96335,0.0,1.94,445.67,2.43,519.46,2.30,0.0,1.94,504.67,2.39,...,0.0,2.89,287.12,2.54,0.0,1.94,0.0,2.90,253.27,2.58


#### Prior known covariates (9)

In [12]:
# non_stages = data[['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'FLOW_S25B', 'GATE_S25B', 'FLOW_S26', 'GATE_S26', 'PUMP_S26', 'PUMP_S25B', 'MEAN_RAIN']]
prior_known_cov = data[['WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2', 'PUMP_S25B', 'GATE_S26_1', 'GATE_S26_2', 'PUMP_S26', 'MEAN_RAIN']]
print("prior_known_cov.shape:", prior_known_cov.shape)

prior_known_cov_supervised = series_to_supervised(prior_known_cov, n_hours, K)
print("prior_known_cov_supervised.shape:", prior_known_cov_supervised.shape)

prior_known_cov.shape: (96432, 9)
prior_known_cov_supervised.shape: (96337, 864)


In [13]:
prior_known_cov_supervised.reset_index(drop=True, inplace=True)
prior_known_cov_supervised.columns = ['past_known_cov_' + i for i in list(prior_known_cov_supervised.columns)]

In [14]:
prior_known_cov_supervised

,past_known_cov_var1(t-72),past_known_cov_var2(t-72),past_known_cov_var3(t-72),past_known_cov_var4(t-72),past_known_cov_var5(t-72),past_known_cov_var6(t-72),past_known_cov_var7(t-72),past_known_cov_var8(t-72),past_known_cov_var9(t-72),past_known_cov_var1(t-71),...,past_known_cov_var9(t+22),past_known_cov_var1(t+23),past_known_cov_var2(t+23),past_known_cov_var3(t+23),past_known_cov_var4(t+23),past_known_cov_var5(t+23),past_known_cov_var6(t+23),past_known_cov_var7(t+23),past_known_cov_var8(t+23),past_known_cov_var9(t+23)
0,0.34,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,-0.28,...,0.0,1.99,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,-0.28,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,-0.73,...,0.0,2.17,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,-0.73,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,-0.85,...,0.0,2.06,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,-0.85,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,-0.49,...,0.0,1.50,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,-0.49,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.21,...,0.0,0.86,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96332,1.79,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.000000,1.31,...,0.0,1.19,0.0,0.0,0.0,0.0,0.0,2.61,0.0,0.0
96333,1.31,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.000000,0.86,...,0.0,1.61,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0
96334,0.86,0.0,0.0,2.58,0.0,0.0,2.57,0.0,0.000000,0.42,...,0.0,1.90,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0
96335,0.42,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.008750,-0.01,...,0.0,2.03,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0


In [15]:
prior_known_future_cov = prior_known_cov_supervised.iloc[:, n_hours*prior_known_cov.shape[1]:]
prior_known_future_cov

,past_known_cov_var1(t),past_known_cov_var2(t),past_known_cov_var3(t),past_known_cov_var4(t),past_known_cov_var5(t),past_known_cov_var6(t),past_known_cov_var7(t),past_known_cov_var8(t),past_known_cov_var9(t),past_known_cov_var1(t+1),...,past_known_cov_var9(t+22),past_known_cov_var1(t+23),past_known_cov_var2(t+23),past_known_cov_var3(t+23),past_known_cov_var4(t+23),past_known_cov_var5(t+23),past_known_cov_var6(t+23),past_known_cov_var7(t+23),past_known_cov_var8(t+23),past_known_cov_var9(t+23)
0,2.03,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,1.47,...,0.0,1.99,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,1.47,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.83,...,0.0,2.17,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,0.83,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.31,...,0.0,2.06,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,0.31,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-0.18,...,0.0,1.50,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,-0.18,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-0.55,...,0.0,0.86,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96332,1.94,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0,2.08,...,0.0,1.19,0.0,0.0,0.0,0.0,0.0,2.61,0.0,0.0
96333,2.08,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0,1.92,...,0.0,1.61,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0
96334,1.92,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0,1.55,...,0.0,1.90,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0
96335,1.55,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0,1.08,...,0.0,2.03,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0


In [16]:
prior_known_future_cov.shape

(96337, 216)

### N_out, N_in, K

In [17]:
# specify the number of lag hours
n_features = stages.shape[1] + prior_unknown_cov.shape[1] + prior_known_cov.shape[1]

In [18]:
n_features

19

### Concatenation

In [19]:
past_prior_unknown_cov_supervised = prior_unknown_cov_supervised.iloc[:, :n_hours*prior_unknown_cov.shape[1]]
past_prior_known_cov_supervised = prior_known_cov_supervised.iloc[:, :n_hours*prior_known_cov.shape[1]]

In [20]:
# prior_known_cov_supervised

In [21]:
columns = []
for i in range(n_hours):
    columns = columns + past_prior_unknown_cov_supervised.columns[i*prior_unknown_cov.shape[1]:(i+1)*prior_unknown_cov.shape[1]].tolist()
    columns = columns + past_prior_known_cov_supervised.columns[i*prior_known_cov.shape[1]:(i+1)*prior_known_cov.shape[1]].tolist()
# columns

In [22]:
past_unknown_known_cov = concat([past_prior_unknown_cov_supervised, past_prior_known_cov_supervised],
                                axis=1)

In [23]:
past_unknown_known_cov

,past_unknown_cov_var1(t-72),past_unknown_cov_var2(t-72),past_unknown_cov_var3(t-72),past_unknown_cov_var4(t-72),past_unknown_cov_var5(t-72),past_unknown_cov_var6(t-72),past_unknown_cov_var1(t-71),past_unknown_cov_var2(t-71),past_unknown_cov_var3(t-71),past_unknown_cov_var4(t-71),...,past_known_cov_var9(t-2),past_known_cov_var1(t-1),past_known_cov_var2(t-1),past_known_cov_var3(t-1),past_known_cov_var4(t-1),past_known_cov_var5(t-1),past_known_cov_var6(t-1),past_known_cov_var7(t-1),past_known_cov_var8(t-1),past_known_cov_var9(t-1)
0,0.0,2.17,0.00,2.84,0.00,2.78,0.0,2.17,0.00,2.84,...,0.0,2.13,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
1,0.0,2.17,0.00,2.84,0.00,2.78,0.0,2.17,0.00,2.84,...,0.0,2.03,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
2,0.0,2.17,0.00,2.84,0.00,2.79,0.0,2.15,0.00,2.83,...,0.0,1.47,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
3,0.0,2.15,0.00,2.83,0.00,2.79,0.0,2.13,0.00,2.84,...,0.0,0.83,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
4,0.0,2.13,0.00,2.84,0.00,2.79,0.0,2.12,0.00,2.84,...,0.0,0.31,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96332,0.0,1.92,240.09,2.52,302.67,2.55,0.0,1.93,325.17,2.51,...,0.0,1.57,0.0,0.0,2.56,0.0,0.0,2.57,0.0,0.0
96333,0.0,1.93,325.17,2.51,393.59,2.48,0.0,1.94,406.56,2.48,...,0.0,1.94,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0
96334,0.0,1.94,406.56,2.48,475.32,2.40,0.0,1.94,445.67,2.43,...,0.0,2.08,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0
96335,0.0,1.94,445.67,2.43,519.46,2.30,0.0,1.94,504.67,2.39,...,0.0,1.92,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0


In [24]:
# non_stages_supervised.reset_index(drop=True, inplace=True)
# stages_supervised.reset_index(drop=True, inplace=True)

all_data = concat([past_unknown_known_cov, stages_supervised.iloc[:, :]], axis=1)

In [25]:
# print("all_data", all_data)
print("all_data.shape:", all_data.shape)

all_data.shape: (96337, 1464)


In [26]:
# all_data

### Train & Test set

In [27]:
all_data = all_data.values
future_covariates = prior_known_future_cov.values

In [28]:
n_train_hours = int(len(data)*0.8)
print("n_train_hours:", n_train_hours)


train = all_data[:n_train_hours, :]    # 0 column is the rainfall to measure heavy/medium/light
test = all_data[n_train_hours:, :]

n_train_hours: 77145


In [29]:
train_future_cov = prior_known_future_cov.iloc[:n_train_hours, :]
test_future_cov = prior_known_future_cov.iloc[n_train_hours:, :] 

In [30]:
test_future_cov.shape

(19192, 216)

### Normalization

In [31]:
n_hours

72

In [32]:
n_features

19

In [33]:
# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, n_obs:]
test_X, test_y = test[:, :n_obs], test[:, n_obs:]
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape", train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)
test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

train_X.shape, train_y.shape, test_X.shape, test_y.shape (77145, 1368) (77145, 96) (19192, 1368) (19192, 96)


In [34]:
# normalize features
scaler1 = MinMaxScaler(feature_range=(0, 1))

train_future_cov = scaler1.fit_transform(train_future_cov)
test_future_cov = scaler1.fit_transform(test_future_cov)
print("train_future_cov.shape, test_future_cov.shape:", train_future_cov.shape, test_future_cov.shape)

train_future_cov.shape, test_future_cov.shape: (77145, 216) (19192, 216)


In [35]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape: \n", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

train_X.shape, train_y.shape, test_X.shape, test_y.shape: 
 (77145, 72, 19) (77145, 96) (19192, 72, 19) (19192, 96)


In [36]:
train_future_cov = train_future_cov.reshape(train_future_cov.shape[0], K, -1)
test_future_cov = test_future_cov.reshape(test_future_cov.shape[0], K, -1)
print("train_future_cov.shape, test_future_cov.shape:", train_future_cov.shape, test_future_cov.shape)

train_future_cov.shape, test_future_cov.shape: (77145, 24, 9) (19192, 24, 9)


In [37]:
# train_y = train_y.reshape((train_y.shape[0], K, -1))
# test_y = test_y.reshape((test_y.shape[0], K, -1))
# print(train_y.shape, test_y.shape)

### Model

In [39]:
# Encoding the past
past_inputs = tf.keras.Input(shape=(n_hours, n_features), name='past_inputs')
encoder = tf.keras.layers.SimpleRNN(128, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(past_inputs)
encoder = tf.keras.layers.SimpleRNN(64, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(encoder)
encoder = tf.keras.layers.SimpleRNN(32, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(encoder)
encoder = tf.keras.layers.SimpleRNN(9, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(encoder)


# Combine past + future
future_inputs = tf.keras.Input(shape=(K, prior_known_cov.shape[1]), name='future_inputs')

past_future_input = concatenate([encoder, future_inputs], axis=1)


# decoder
decoder = tf.keras.layers.SimpleRNN(128, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(past_future_input)
decoder = tf.keras.layers.SimpleRNN(64, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(decoder)
decoder = tf.keras.layers.SimpleRNN(32, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(decoder)
decoder = tf.keras.layers.SimpleRNN(16, 
                                    activation='relu', 
                                    return_sequences=True,
                                    recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                                    kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(decoder)

fc = layers.Flatten()(decoder)

# final output
fc = tf.keras.layers.Dense(256, activation='relu')(fc)
fc = tf.keras.layers.Dense(128, activation='relu')(fc)
fc = tf.keras.layers.Dense(64, activation='relu')(fc)
output = tf.keras.layers.Dense(train_y.shape[1])(fc)

model = tf.keras.models.Model(inputs=[past_inputs, future_inputs], outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
past_inputs (InputLayer)        [(None, 72, 19)]     0                                            
__________________________________________________________________________________________________
simple_rnn_8 (SimpleRNN)        (None, 72, 128)      18944       past_inputs[0][0]                
__________________________________________________________________________________________________
simple_rnn_9 (SimpleRNN)        (None, 72, 64)       12352       simple_rnn_8[0][0]               
__________________________________________________________________________________________________
simple_rnn_10 (SimpleRNN)       (None, 72, 32)       3104        simple_rnn_9[0][0]               
____________________________________________________________________________________________

In [40]:
lr = 0.00001
EPOCHS = 6000

lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=5e-4, 
                                                          decay_steps=10000,
                                                          decay_rate=0.99)


model.compile(
#               optimizer=Adam(learning_rate=lr, decay=lr/EPOCHS),
                optimizer=Adam(learning_rate=lr_schedule),
#               optimizer='adam',
              loss='mse',
              metrics=['mae'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=1000)
mc = ModelCheckpoint('saved_model/rnn_encoder_decoder_24h_ws_cp1.h5', monitor='val_mae', mode='min', verbose=2, save_best_only=True)

history = model.fit(x=[train_X, train_future_cov], y=train_y, 
                    epochs=EPOCHS,
                    verbose=2,
                    validation_data=([test_X, test_future_cov], test_y),
                    batch_size = 512,
                    callbacks=[es, mc]
                   )

plt.rcParams["figure.figsize"] = (8, 6)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(fontsize=14)
plt.title("Training loss vds Testing loss", fontsize=18)
# plt.savefig('graph/rnn_loss.png', dpi=300)
plt.show()

Epoch 1/6000


2023-02-04 19:19:06.660539: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10



Epoch 00001: val_mae improved from inf to 0.05660, saving model to saved_model/rnn_encoder_decoder_24h_ws_cp1.h5
151/151 - 79s - loss: 0.0807 - mae: 0.1034 - val_loss: 0.0545 - val_mae: 0.0566
Epoch 2/6000

Epoch 00002: val_mae improved from 0.05660 to 0.03785, saving model to saved_model/rnn_encoder_decoder_24h_ws_cp1.h5
151/151 - 77s - loss: 0.0428 - mae: 0.0229 - val_loss: 0.0375 - val_mae: 0.0378
Epoch 3/6000

Epoch 00003: val_mae improved from 0.03785 to 0.03689, saving model to saved_model/rnn_encoder_decoder_24h_ws_cp1.h5
151/151 - 76s - loss: 0.0314 - mae: 0.0180 - val_loss: 0.0290 - val_mae: 0.0369
Epoch 4/6000

Epoch 00004: val_mae improved from 0.03689 to 0.03655, saving model to saved_model/rnn_encoder_decoder_24h_ws_cp1.h5
151/151 - 77s - loss: 0.0247 - mae: 0.0172 - val_loss: 0.0237 - val_mae: 0.0366
Epoch 5/6000

Epoch 00005: val_mae improved from 0.03655 to 0.03462, saving model to saved_model/rnn_encoder_decoder_24h_ws_cp1.h5
151/151 - 76s - loss: 0.0202 - mae: 0.0162

Epoch 49/6000

Epoch 00049: val_mae did not improve from 0.02739
151/151 - 79s - loss: 6.7517e-04 - mae: 0.0111 - val_loss: 0.0016 - val_mae: 0.0286
Epoch 50/6000

Epoch 00050: val_mae improved from 0.02739 to 0.02711, saving model to saved_model/rnn_encoder_decoder_24h_ws_cp1.h5
151/151 - 80s - loss: 6.6682e-04 - mae: 0.0110 - val_loss: 0.0015 - val_mae: 0.0271
Epoch 51/6000

Epoch 00051: val_mae did not improve from 0.02711
151/151 - 78s - loss: 6.5812e-04 - mae: 0.0110 - val_loss: 0.0015 - val_mae: 0.0280
Epoch 52/6000

Epoch 00052: val_mae did not improve from 0.02711
151/151 - 78s - loss: 6.5230e-04 - mae: 0.0110 - val_loss: 0.0015 - val_mae: 0.0283
Epoch 53/6000

Epoch 00053: val_mae did not improve from 0.02711
151/151 - 79s - loss: 6.4759e-04 - mae: 0.0111 - val_loss: 0.0015 - val_mae: 0.0282
Epoch 54/6000

Epoch 00054: val_mae did not improve from 0.02711
151/151 - 78s - loss: 6.3719e-04 - mae: 0.0110 - val_loss: 0.0018 - val_mae: 0.0312
Epoch 55/6000

Epoch 00055: val_mae did

Epoch 104/6000

Epoch 00104: val_mae did not improve from 0.02711
151/151 - 88s - loss: 4.9700e-04 - mae: 0.0108 - val_loss: 0.0014 - val_mae: 0.0281
Epoch 105/6000

Epoch 00105: val_mae did not improve from 0.02711
151/151 - 88s - loss: 4.9608e-04 - mae: 0.0108 - val_loss: 0.0014 - val_mae: 0.0282
Epoch 106/6000

Epoch 00106: val_mae did not improve from 0.02711
151/151 - 87s - loss: 4.9561e-04 - mae: 0.0108 - val_loss: 0.0015 - val_mae: 0.0289
Epoch 107/6000

Epoch 00107: val_mae did not improve from 0.02711
151/151 - 85s - loss: 4.9430e-04 - mae: 0.0108 - val_loss: 0.0014 - val_mae: 0.0282
Epoch 108/6000

Epoch 00108: val_mae did not improve from 0.02711
151/151 - 84s - loss: 4.9101e-04 - mae: 0.0108 - val_loss: 0.0015 - val_mae: 0.0298
Epoch 109/6000

Epoch 00109: val_mae did not improve from 0.02711
151/151 - 84s - loss: 4.9021e-04 - mae: 0.0108 - val_loss: 0.0015 - val_mae: 0.0291
Epoch 110/6000

Epoch 00110: val_mae did not improve from 0.02711
151/151 - 85s - loss: 4.8998e-04 -

...

Epoch 1479/6000

Epoch 01479: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4875e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0282
Epoch 1480/6000

Epoch 01480: val_mae did not improve from 0.02688
151/151 - 47s - loss: 3.4745e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0285
Epoch 1481/6000

Epoch 01481: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4755e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0280
Epoch 1482/6000

Epoch 01482: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4736e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0281
Epoch 1483/6000

Epoch 01483: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4718e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0285
Epoch 1484/6000

Epoch 01484: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4820e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0283
Epoch 1485/6000

Epoch 01485: val_mae did not improve from 0.02688
151/151 - 47s - loss: 3.469

Epoch 1534/6000

Epoch 01534: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4779e-04 - mae: 0.0096 - val_loss: 0.0012 - val_mae: 0.0272
Epoch 1535/6000

Epoch 01535: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4674e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0281
Epoch 1536/6000

Epoch 01536: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4701e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0286
Epoch 1537/6000

Epoch 01537: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4761e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0277
Epoch 1538/6000

Epoch 01538: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4652e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0284
Epoch 1539/6000

Epoch 01539: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.4666e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0281
Epoch 1540/6000

Epoch 01540: val_mae did not improve from 0.02688
151/151 - 46s - loss: 3.467

Epoch 1589/6000

Epoch 01589: val_mae did not improve from 0.02688
151/151 - 52s - loss: 3.4719e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0281
Epoch 1590/6000

Epoch 01590: val_mae did not improve from 0.02688
151/151 - 53s - loss: 3.4629e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0277
Epoch 1591/6000

Epoch 01591: val_mae did not improve from 0.02688
151/151 - 53s - loss: 3.4583e-04 - mae: 0.0095 - val_loss: 0.0013 - val_mae: 0.0285
Epoch 1592/6000

Epoch 01592: val_mae did not improve from 0.02688
151/151 - 53s - loss: 4.0653e-04 - mae: 0.0107 - val_loss: 0.0016 - val_mae: 0.0310
Epoch 1593/6000

Epoch 01593: val_mae did not improve from 0.02688
151/151 - 52s - loss: 5.1086e-04 - mae: 0.0132 - val_loss: 0.0013 - val_mae: 0.0278
Epoch 1594/6000

Epoch 01594: val_mae did not improve from 0.02688
151/151 - 52s - loss: 3.4639e-04 - mae: 0.0096 - val_loss: 0.0012 - val_mae: 0.0277
Epoch 1595/6000

Epoch 01595: val_mae did not improve from 0.02688
151/151 - 52s - loss: 3.455

Epoch 1644/6000

Epoch 01644: val_mae did not improve from 0.02688
151/151 - 50s - loss: 3.4590e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0285
Epoch 1645/6000

Epoch 01645: val_mae did not improve from 0.02688
151/151 - 50s - loss: 3.4592e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0280
Epoch 1646/6000

Epoch 01646: val_mae did not improve from 0.02688
151/151 - 50s - loss: 3.4560e-04 - mae: 0.0095 - val_loss: 0.0013 - val_mae: 0.0284
Epoch 1647/6000

Epoch 01647: val_mae did not improve from 0.02688
151/151 - 51s - loss: 3.4605e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0282
Epoch 1648/6000

Epoch 01648: val_mae did not improve from 0.02688
151/151 - 50s - loss: 3.4550e-04 - mae: 0.0095 - val_loss: 0.0014 - val_mae: 0.0293
Epoch 1649/6000

Epoch 01649: val_mae did not improve from 0.02688
151/151 - 53s - loss: 3.4574e-04 - mae: 0.0096 - val_loss: 0.0013 - val_mae: 0.0282
Epoch 1650/6000

Epoch 01650: val_mae did not improve from 0.02688
151/151 - 54s - loss: 3.457

KeyboardInterrupt: 

### Prediction

In [41]:
from tensorflow.keras.models import load_model

model = load_model('saved_model/rnn_encoder_decoder_24h_ws_cp1.h5')

In [42]:
y_hat = model.predict([test_X, test_future_cov])

In [43]:
print(y_hat.shape)
print(test_y.shape)

(19192, 96)
(19192, 96)


In [44]:
inv_yhat = scaler.inverse_transform(y_hat)
inv_y = scaler.inverse_transform(test_y)

inv_yhat = pd.DataFrame(inv_yhat)
inv_y = pd.DataFrame(inv_y)
print("inv_y.shape, inv_yhat.shape", inv_y.shape, inv_yhat.shape)

inv_y.shape, inv_yhat.shape (19192, 96) (19192, 96)


### Performance

In [45]:
print('MAE = {}'.format(float("{:.3f}".format(mae(inv_yhat.iloc[:, :], inv_y.iloc[:, :])))))
print('RMSE = {}'.format(float("{:.3f}".format(sqrt(mse(inv_yhat.iloc[:, :], inv_y.iloc[:, :]))))))

MAE = 0.142
RMSE = 0.177
